Task 1

In [3]:
import heapq

def get_neighbors(maze, node):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    neighbors = []
    for dx, dy in directions:
        x, y = node[0] + dx, node[1] + dy
        if 0 <= x < len(maze) and 0 <= y < len(maze[0]) and maze[x][y] != '#':  # Check boundaries & walls
            neighbors.append((x, y))
    return neighbors

def heuristic(pos, goals):
    if not goals:
        return 0
    return min(abs(pos[0] - g[0]) + abs(pos[1] - g[1]) for g in goals)

def a_star_multi_goal(maze, start, goals):
    pq = []  # Priority queue
    visited = set()
    start_state = (0, start, frozenset())  # (cost, position, visited_goals)

    heapq.heappush(pq, start_state)

    while pq:
        cost, current, visited_goals = heapq.heappop(pq)


        if visited_goals == frozenset(goals):
            return cost

        if (current, visited_goals) in visited:
            continue
        visited.add((current, visited_goals))

        for neighbor in get_neighbors(maze, current):
            new_visited_goals = visited_goals | {neighbor} if neighbor in goals else visited_goals
            new_cost = cost + 1
            priority = new_cost + heuristic(neighbor, goals - new_visited_goals)
            heapq.heappush(pq, (priority, neighbor, new_visited_goals))

    return -1


maze = [
    ['S', '.', '.', '#', 'G'],
    ['.', '#', '.', '.', '.'],
    ['.', '.', '#', 'G', '.'],
    ['#', '.', '.', '.', '.'],
    ['G', '.', '#', '.', 'G']
]


start = None
goals = set()
for i in range(len(maze)):
    for j in range(len(maze[0])):
        if maze[i][j] == 'S':
            start = (i, j)
        elif maze[i][j] == 'G':
            goals.add((i, j))


result = a_star_multi_goal(maze, start, goals)
print("Shortest Path Length Visiting All Goals:", result)


Shortest Path Length Visiting All Goals: 54


task 2: Q2. Implement an A* Search where the edge costs change dynamically at random
intervals. The algorithm should adapt to these changes and always find the
optimal path. Recompute and adjust paths in real time without restarting the
algorithm from scratch.

In [6]:
import heapq
import random

class DynamicAStar:
    def __init__(self, grid, start, goal):
        self.grid = grid
        self.start = start
        self.goal = goal
        self.rows = len(grid)
        self.cols = len(grid[0])

    def heuristic(self, pos):
        return abs(pos[0] - self.goal[0]) + abs(pos[1] - self.goal[1])

    def get_neighbors(self, node):
        directions = [(0,1), (1,0), (0,-1), (-1,0)]
        neighbors = []
        for dx, dy in directions:
            x, y = node[0] + dx, node[1] + dy
            if 0 <= x < self.rows and 0 <= y < self.cols and self.grid[x][y] != '#':
                neighbors.append((x, y))
        return neighbors

    def a_star(self):
        open_set = [(0, self.start)]
        g_costs = {self.start: 0}
        came_from = {}

        while open_set:
            _, current = heapq.heappop(open_set)

            if current == self.goal:
                return self.reconstruct_path(came_from)

            for neighbor in self.get_neighbors(current):
                new_cost = g_costs[current] + self.grid[neighbor[0]][neighbor[1]]

                if neighbor not in g_costs or new_cost < g_costs[neighbor]:
                    g_costs[neighbor] = new_cost
                    f_cost = new_cost + self.heuristic(neighbor)
                    heapq.heappush(open_set, (f_cost, neighbor))
                    came_from[neighbor] = current

        return None

    def reconstruct_path(self, came_from):
        path = []
        node = self.goal
        while node in came_from:
            path.append(node)
            node = came_from[node]
        path.append(self.start)
        return path[::-1]

    def change_edge_costs(self):
        for _ in range(2):  # Change 2 random edges
            x, y = random.randint(0, self.rows-1), random.randint(0, self.cols-1)
            if self.grid[x][y] not in ['#', self.start, self.goal]:
                self.grid[x][y] = random.randint(1, 9)

        print("Edge costs updated!")
        return self.a_star()  # Recompute path


grid = [
    [1, 1, 1, '#', 1],
    [1, '#', 1, 1, 1],
    [1, 1, '#', 1, 1],
    ['#', 1, 1, 1, 1],
    [1, 1, '#', 1, 1]
]

start = (0, 0)
goal = (4, 4)

astar = DynamicAStar(grid, start, goal)
path = astar.a_star()
print("Initial Path:", path)

new_path = astar.change_edge_costs()
print("Updated Path:", new_path)


Initial Path: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4)]
Edge costs updated!
Updated Path: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 3), (4, 3), (4, 4)]


Q3. Delivery Route Optimization with Time Windows
● Description: Using the Greedy Best-First Search, optimize delivery routes for a
set of delivery points. Each delivery point has a specific time window for delivery,
and the algorithm must prioritize those with stricter deadlines.
● Challenge: Ensure that the algorithm handles time constraints efficiently while
minimizing total travel distance.

In [5]:
import heapq

class DeliveryRouteOptimizer:
    def __init__(self, start, deliveries):
        self.start = start
        self.deliveries = deliveries
        self.current_time = 0
        self.route = []

    def distance(self, point1, point2):
        return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])

    def greedy_best_first_search(self):
        priority_queue = []


        for d in self.deliveries:
            x, y, t_start, t_end = d
            heapq.heappush(priority_queue, (t_end, x, y, t_start, t_end))  # (priority, x, y, t_start, t_end)

        current_location = self.start

        while priority_queue:
            possible_deliveries = []


            while priority_queue:
                t_end, x, y, t_start, t_end = heapq.heappop(priority_queue)
                travel_time = self.distance(current_location, (x, y))


                if self.current_time + travel_time <= t_end:
                    possible_deliveries.append((travel_time, x, y, t_start, t_end))

            if not possible_deliveries:
                break  # No more feasible deliveries


            possible_deliveries.sort()
            travel_time, x, y, t_start, t_end = possible_deliveries[0]


            self.current_time += travel_time
            current_location = (x, y)
            self.route.append((x, y, self.current_time))  # Store completed delivery


            self.deliveries = [d for d in self.deliveries if (d[0], d[1]) != (x, y)]


            priority_queue = []
            for d in self.deliveries:
                x, y, t_start, t_end = d
                heapq.heappush(priority_queue, (t_end, x, y, t_start, t_end))

        return self.route

start_location = (0, 0)  # Warehouse
deliveries = [
    (2, 3, 2, 10),
    (5, 1, 4, 8),
    (1, 6, 1, 7),
    (4, 4, 3, 12),
    (3, 2, 2, 9)
]


optimizer = DeliveryRouteOptimizer(start_location, deliveries)
optimized_route = optimizer.greedy_best_first_search()

print("\nOptimized Delivery Route (x, y, arrival time):")
for stop in optimized_route:
    print(stop)



Optimized Delivery Route (x, y, arrival time):
(2, 3, 5)
(3, 2, 7)
(4, 4, 10)
